#Data cleaning

In [16]:
import pandas as pd

In [1]:
!pip install accelerate>=0.20.1

In [2]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.7 MB/s eta 0:00:00


In [3]:
import accelerate
import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer

In [ ]:
data = pd.read_excel("/content/Датасет.xlsx")

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
#Функция для очистки датасета по условию: если поле, которое нужно заполнить, не содержится в исходном поле (требования к соискателю), этот элемент удаляется из датасета
def contains(str1, str2):
  if str1.find(str2) != -1:
    return True
  else:
    return False
def clear_nan(string):
  if str(string) == "nan":
    return ""
  else:
    return str(string)

In [ ]:
data["requirements(Требования к соискателю)"] = data.apply(lambda x: clear_nan(x["requirements(Требования к соискателю)"]), axis=1)
data["terms(Условия)"] = data.apply(lambda x: clear_nan(x["terms(Условия)"]), axis=1)
data["notes(Примечания)"] = data.apply(lambda x: clear_nan(x["notes(Примечания)"]), axis=1)

In [ ]:
data["Содержит требования"] = data.apply(lambda x: contains(x["responsibilities(Должностные обязанности)"], x["requirements(Требования к соискателю)"]), axis=1)
data["Содержит условия"] = data.apply(lambda x: contains(x["responsibilities(Должностные обязанности)"], x["terms(Условия)"]) , axis=1)
data["Содержит примечания"] = data.apply(lambda x: contains(x["responsibilities(Должностные обязанности)"], x["notes(Примечания)"]) , axis=1)

In [ ]:
data_v1 = data[(data["Содержит примечания"]) & (data["Содержит условия"]) & (data["Содержит требования"])]

In [ ]:
#Полученный размер датасета
len(data_v1)

873

#Tagging

In [ ]:
#Настройка функции очистки, удаляющей из строки все символы, кроме допустимых. Необходимо для удаления эмодзи и т.д.
import re
regex = re.compile(r"[^ .,-?!:0123456789а-яА-Я]")

In [ ]:
string = " .абв?10#"
regex.sub("", string)

' .абв?10'

In [ ]:
#Очистка полей от недопустимых символов
data_v1["Clear responsibilities"] = data_v1.apply(lambda x: regex.sub("", x["responsibilities(Должностные обязанности)"]), axis=1)
data_v1["Clear requirements"] = data_v1.apply(lambda x: regex.sub("", x["requirements(Требования к соискателю)"]), axis=1)
data_v1["Clear terms"] = data_v1.apply(lambda x: regex.sub("", x["terms(Условия)"]), axis=1)
data_v1["Clear notes"] = data_v1.apply(lambda x: regex.sub("", x["notes(Примечания)"]), axis=1)

<ipython-input-17-607c396c4acb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_v1["Clear responsibilities"] = data_v1.apply(lambda x: regex.sub("", x["responsibilities(Должностные обязанности)"]), axis=1)
<ipython-input-17-607c396c4acb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_v1["Clear requirements"] = data_v1.apply(lambda x: regex.sub("", x["requirements(Требования к соискателю)"]), axis=1)
<ipython-input-17-607c396c4acb>:3: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [ ]:
data_v1["Содержит требования"] = data_v1.apply(lambda x: contains(x["Clear responsibilities"], x["Clear requirements"]), axis=1)
data_v1["Содержит условия"] = data_v1.apply(lambda x: contains(x["Clear responsibilities"], x["Clear terms"]) , axis=1)
data_v1["Содержит примечания"] = data_v1.apply(lambda x: contains(x["Clear responsibilities"], x["Clear notes"]) , axis=1)

<ipython-input-18-53a5841fb500>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_v1["Содержит требования"] = data_v1.apply(lambda x: contains(x["Clear responsibilities"], x["Clear requirements"]), axis=1)
<ipython-input-18-53a5841fb500>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_v1["Содержит условия"] = data_v1.apply(lambda x: contains(x["Clear responsibilities"], x["Clear terms"]) , axis=1)
<ipython-input-18-53a5841fb500>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [ ]:
#Определение классов для разметки текста на задаче NER
labels = ["requirements", "terms", "notes"]

In [26]:
#Функция для преобразования датасета к задаче NER, присвоение класса каждому токену
def get_tokens_with_entities(text, subtexts):
  spaces = [m.start() for m in re.finditer(' ', text)]
  starts = []
  strings = []
  for label in labels:
    string = subtexts[label]
    strings.append(string)
    starts.append(text.find(string))
  tags = "0"*len(text)
  for ind, i in enumerate(starts):
    tags = tags[:i] + str(ind+1)*len(strings[ind]) + tags[i + len(strings[ind]):]
  t_list = list(tags)
  for i in spaces:
    t_list[i] = " "
  tags = "".join(t_list)
  tokens_list = text.split(" ")
  tags_list = tags.split(" ")
  tokens = []
  tags = []
  for ind, i in enumerate(tokens_list):
    if i != "":
      tokens.append(i)
      tags.append(tags_list[ind])
  good_tags = []
  for tag in tags:
    good_tags.append(int(tag[0]))
  output = []
  for ind, i in enumerate(tokens):
    label_ind = good_tags[ind] - 1
    label = "O"
    if label_ind != -1:
      label = labels[label_ind]
    cort = ()
    if label == "O":
      cort = (i, label)
    else:
      if ind == 0:
        cort = (i, "B-" + label)
      else:
        if output[ind-1][1] != "O":
          if output[ind-1][1][2:] == label:
            cort = (i, "I-" + label)
          else:
            cort = (i, "B-" + label)
        else:
          cort = (i, "B-" + label)
    output.append(cort)
  return output

In [ ]:
#Создание словаря классов и полей им соответствующих
subtexts = []
for ind in range(len(data_v1)):
  subtext = {}
  subtext["requirements"] = data_v1.iloc[ind]["Clear requirements"]
  subtext["terms"] = data_v1.iloc[ind]["Clear terms"]
  subtext["notes"] = data_v1.iloc[ind]["Clear notes"]
  subtexts.append(subtext)

In [ ]:
subtexts[3]

{'requirements': 'Выполнение работ по гнутью и резке арматурной стали на ручных, электромеханических и электрических станках',
 'terms': '',
 'notes': ''}

In [ ]:
#Преобразование датасета
marked = []
for ind in range(len(data_v1)):
  vacancy = data_v1.iloc[ind]
  entities = get_tokens_with_entities(vacancy["Clear responsibilities"], subtexts[ind])
  marked.append(entities)

In [ ]:
len(subtexts)

873

#NER training

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.6 MB/s eta 0:00:00


In [ ]:
#Класс для окончательного преобразования датасета для подачи в модель
class NERDataMaker:
    def __init__(self, texts, entities):
        self.unique_entities = []
        self.processed_texts = []

        temp_processed_texts = []
        for ind, text in enumerate(texts):
            tokens_with_entities = get_tokens_with_entities(text, entities[ind])
            for _, ent in tokens_with_entities:
                if ent not in self.unique_entities:
                    self.unique_entities.append(ent)
            temp_processed_texts.append(tokens_with_entities)

        self.unique_entities.sort(key=lambda ent: ent if ent != "O" else "")

        for tokens_with_entities in temp_processed_texts:
            self.processed_texts.append([(t, self.unique_entities.index(ent)) for t, ent in tokens_with_entities])

    @property
    def id2label(self):
        return dict(enumerate(self.unique_entities))

    @property
    def label2id(self):
        return {v:k for k, v in self.id2label.items()}

    def __len__(self):
        return len(self.processed_texts)

    def __getitem__(self, idx):
        def _process_tokens_for_one_text(id, tokens_with_encoded_entities):
            ner_tags = []
            tokens = []
            for t, ent in tokens_with_encoded_entities:
                ner_tags.append(ent)
                tokens.append(t)

            return {
                "id": id,
                "ner_tags": ner_tags,
                "tokens": tokens
            }

        tokens_with_encoded_entities = self.processed_texts[idx]
        if isinstance(idx, int):
            return _process_tokens_for_one_text(idx, tokens_with_encoded_entities)
        else:
            return [_process_tokens_for_one_text(i+idx.start, tee) for i, tee in enumerate(tokens_with_encoded_entities)]

    def as_hf_dataset(self, tokenizer):
        from datasets import Dataset, Features, Value, ClassLabel, Sequence
        def tokenize_and_align_labels(examples):
            tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

            labels = []
            for i, label in enumerate(examples[f"ner_tags"]):
                word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
                previous_word_idx = None
                label_ids = []
                for word_idx in word_ids:  # Set the special tokens to -100.
                    if word_idx is None:
                        label_ids.append(-100)
                    elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                        label_ids.append(label[word_idx])
                    else:
                        label_ids.append(-100)
                    previous_word_idx = word_idx
                labels.append(label_ids)

            tokenized_inputs["labels"] = labels
            return tokenized_inputs

        ids, ner_tags, tokens = [], [], []
        for i, pt in enumerate(self.processed_texts):
            ids.append(i)
            pt_tokens,pt_tags = list(zip(*pt))
            ner_tags.append(pt_tags)
            tokens.append(pt_tokens)
        data = {
            "id": ids,
            "ner_tags": ner_tags,
            "tokens": tokens
        }
        features = Features({
            "tokens": Sequence(Value("string")),
            "ner_tags": Sequence(ClassLabel(names=dm.unique_entities)),
            "id": Value("int32")
        })
        ds = Dataset.from_dict(data, features)
        tokenized_ds = ds.map(tokenize_and_align_labels, batched=True)
        return tokenized_ds

In [ ]:
dm = NERDataMaker(list(data_v1["Clear responsibilities"]), subtexts)

In [ ]:
print(f"total examples = {len(dm)}")
print(dm[0:3])

total examples = 873
[{'id': 0, 'ner_tags': [0, 0, 0, 0, 0, 0], 'tokens': ['Работа', 'на', 'строительных', 'площадках.', 'Очистных', 'сооружениях.']}, {'id': 1, 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'tokens': ['Гнутье', 'арматурной', 'стали', 'на', 'механических', 'станках', 'при', 'количестве', 'отгибов', 'на', 'одном', 'стержне', 'более', 'четырех.', 'Сборка', 'и', 'установка', 'сеток', 'и', 'плоских...']}, {'id': 2, 'ner_tags': [0, 0], 'tokens': ['Вязка', 'арматуры.']}]


In [ ]:
#Создание модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained("Evolett/rubert-tiny2-finetuned-ner")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
model = AutoModelForTokenClassification.from_pretrained("Evolett/rubert-tiny2-finetuned-ner", num_labels=len(dm.unique_entities), id2label=dm.id2label, label2id=dm.label2id, ignore_mismatched_sizes=True)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at Evolett/rubert-tiny2-finetuned-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 312]) in the checkpoint and torch.Size([7, 312]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#Определение параметров обучения
training_args = TrainingArguments(
    output_dir="/content/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=40,
    weight_decay=0.01,
)

train_ds = dm.as_hf_dataset(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=train_ds, # eval on training set! ONLY for DEMO!!
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Map:   0%|          | 0/873 [00:00<?, ? examples/s]

In [ ]:
#Обучение модели
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.039958
2,No log,0.857216
3,No log,0.744847
4,No log,0.677684
5,No log,0.629539
6,No log,0.589340
7,No log,0.555751
8,No log,0.528908
9,No log,0.507255
10,0.773100,0.493989


TrainOutput(global_step=2200, training_loss=0.49172000191428444, metrics={'train_runtime': 298.9445, 'train_samples_per_second': 116.811, 'train_steps_per_second': 7.359, 'total_flos': 194165613156216.0, 'train_loss': 0.49172000191428444, 'epoch': 40.0})

#NER inference

In [ ]:
#Пайплайн для инференса модели
from transformers import pipeline
pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

In [ ]:
#Преобразование инференса в удобочитаемый формат
def convert_result(result):
  requirements = ""
  terms = ""
  notes = ""
  for span in result:
    label = span["entity_group"]
    word = span["word"]
    if label == "requirements":
      requirements += word + " "
    elif label == "terms":
      terms += word + " "
    elif label == "notes":
      notes += word + " "
  return requirements, terms, notes

In [ ]:
#Пример входных данных
input_index = 4
input = data_v1.iloc[input_index]["Clear responsibilities"]
input

'Вахта в город Москва.  Обязанности: - армирование каркаса;  Требования: - опыт в строительстве приветствуется; - работа в бригаде;  Условия: - продолжительность вахты 60/30 продление вахты возможно; - Официальное трудоустройство; - ЗП в срок и без задержек; - Авансирование дважды в месяц по 15 000 рублей, 15 и 30 числа; - Питание трехразовое за счет организации; - Выдача спецодежды и Сизов без вычета из заработной платы; - Организованные отправки до объекта покупка билетов; - Помощь в прохождение медицинского осмотра; - Возможность получить квалификационные удостоверения; - Карьерный рост до бригадира/мастера;'

In [ ]:
#Пример выходных данных
result = pipe(input)
convert_result(result)

('опыт в строительстве приветствуется ; - работа в бригаде ',
 'продолжительность вахты 60 / 30 продление вахты возможно ; - Официальное трудоустройство ; - ЗП в срок и без задержек ; - Авансирование дважды в месяц по 15 000 рублей, 15 и 30 числа ; - Питание трехразовое за счет организации ; - Выдача спецодежды и Сизов без вычета из заработной платы ; - Организованные отправки до объекта покупка билетов ; - Помощь в прохождение медицинского осмотра ; - Возможность получить квалификационные удостоверения ; - Карьерный рост до бригадира / мастера ; ',
 '')

In [ ]:
model.save_pretrained("/content/ner_model")

In [ ]:
tokenizer.save_pretrained("/content/ner_model")

('/content/ner_model/tokenizer_config.json',
 '/content/ner_model/special_tokens_map.json',
 '/content/ner_model/vocab.txt',
 '/content/ner_model/added_tokens.json',
 '/content/ner_model/tokenizer.json')

In [ ]:
loaded = AutoModelForTokenClassification.from_pretrained("/content/ner_model").eval().to(device)

In [ ]:
loaded_pipe = pipeline("ner", model=loaded, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

In [ ]:
loaded_result = pipe(input)
convert_result(loaded_result)

('Желание работать и зарабатывать Рассматриваем без опыта работы! всему необходимому научим Обязанности : Бригадная работа под руководством мастера. Монтажные, бетонно - арматурные работы, подсобные виды работ. ',
 'Компенсация затрат на проезд. Проживание бесплатно в квартирах за счет организации, по 2 человека в комнате, в шаговой доступности от производства. Медосмотр на объекте бесплатно! Своевременная и стабильная заработная плата, без задержек ; выплаты строго 2 раза в месяц ; на карту. Фиксированная ставка без скрытых вычетов. Карьерный рост Официальное трудоустройство по ТК РФ. Трудоустройство официальное с первого дня. Питание за счет работодателя Предоставляется спецодежда. БЕЗ вычетов с з / п График вахты выбираете сами 30 / 30, 60 / 30 ',
 'Пока идет строчный набор действует акция ; ПРИВЕДИ ДРУГА ;, премия 4000 руб. Можете сообщить знакомым и друзьям, о наборе специалистов в нашу компанию! Присоединяйтесь к дружному коллективу со стабильной заработной плато , мы быстро раст

In [6]:
#Полный пайплайн обработки вакансии
MODEL_PATH = "/content/model"
INPUT = "Требуются разнорабочие на стройку. Опыт работы не обязателен"
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification
def convert_result(result):
  requirements = ""
  terms = ""
  notes = ""
  for span in result:
    label = span["entity_group"]
    word = span["word"]
    if label == "requirements":
      requirements += word + " "
    elif label == "terms":
      terms += word + " "
    elif label == "notes":
      notes += word + " "
  return requirements, terms, notes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH).eval().to(device)
pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=device)
raw_result = pipe(INPUT)
result = convert_result(raw_result)

In [20]:
data = pd.read_excel("/content/Датасет_old.xlsx")

In [21]:
data["requirements(Требования к соискателю)"] = data.apply(lambda x: convert_result(pipe(x["responsibilities(Должностные обязанности)"]))[0], axis=1)

In [22]:
data["terms(Условия)"] = data.apply(lambda x: convert_result(pipe(x["responsibilities(Должностные обязанности)"]))[1], axis=1)

In [23]:
data["notes(Примечания)"] = data.apply(lambda x: convert_result(pipe(x["responsibilities(Должностные обязанности)"]))[2], axis=1)

In [25]:
data.to_excel("output.xlsx")

In [7]:
result

('Требуются разнорабочие на стройку. Опыт работы не обязателен ', '', '')

#QANER functions

In [ ]:
from collections import namedtuple
from typing import List, Tuple
from tqdm import tqdm, trange
Span = namedtuple(
    "Span", ["token", "label", "start_context_char_pos", "end_context_char_pos"]
)
Instance = namedtuple("Instance", ["context", "question", "answer"])


In [ ]:
import random

import numpy as np
import torch


def set_global_seed(seed: int):
    """
    Set global seed for reproducibility.

    Args:
        seed (int): Seed.
    """

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
from typing import Dict, List
def get_top_valid_spans(
    context_list: List[str],
    question_list: List[str],
    prompt_mapper: Dict[str, str],
    inputs: transformers.tokenization_utils_base.BatchEncoding,
    outputs: transformers.tokenization_utils_base.BatchEncoding,
    offset_mapping_batch: torch.Tensor,
    n_best_size: int = 1,
    max_answer_length: int = 100,
) -> List[List[Span]]:
    """
    Inference function to get top valid spans.
    https://huggingface.co/docs/transformers/tasks/question_answering

    Args:
        context_list (List[str]): context strings in  batch.
        question_list (List[str]): question strings in batch.
        prompt_mapper (Dict[str, str]): prompt mapper.
        inputs (transformers.tokenization_utils_base.BatchEncoding): inputs.
        outputs (transformers.tokenization_utils_base.BatchEncoding): outputs.
        offset_mapping_batch (torch.Tensor): offset mapping.
        n_best_size (int): number of best spans. Defaults to 1.
        max_answer_length (int): max answer length. Defaults to 100.

    Returns:
        List[List[Span]]: top valid spans for each isntance in batch.
    """

    batch_size = len(offset_mapping_batch)

    inv_prompt_mapper = {v: k for k, v in prompt_mapper.items()}

    assert batch_size == len(context_list)
    assert batch_size == len(question_list)
    assert batch_size == len(inputs["input_ids"])
    assert batch_size == len(inputs["token_type_ids"])
    assert batch_size == len(outputs["start_logits"])
    assert batch_size == len(outputs["end_logits"])

    top_valid_spans_batch = []

    # TODO: optimize it
    for i in range(batch_size):
        context = context_list[i]

        offset_mapping = offset_mapping_batch[i].cpu().numpy()
        mask = inputs["token_type_ids"][i].bool().cpu().numpy()
        offset_mapping[~mask] = [0, 0]
        offset_mapping = [
            (span if span != [0, 0] else None) for span in offset_mapping.tolist()
        ]

        start_logits = outputs["start_logits"][i].cpu().numpy()
        end_logits = outputs["end_logits"][i].cpu().numpy()

        start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
        end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
        top_valid_spans = []

        # TODO: validate loop
        for start_index, end_index in zip(start_indexes, end_indexes):
            if (
                start_index >= len(offset_mapping)
                or end_index >= len(offset_mapping)
                or offset_mapping[start_index] is None
                or offset_mapping[end_index] is None
            ):
                continue
            if (end_index < start_index) or (
                end_index - start_index + 1 > max_answer_length
            ):
                continue
            if start_index <= end_index:
                start_context_char_char, end_context_char_char = offset_mapping[
                    start_index
                ]
                span = Span(
                    token=context[start_context_char_char:end_context_char_char],
                    label=inv_prompt_mapper[  # TODO: add inference exception
                        question_list[i].split(r"Какие ")[-1].rstrip(r"?")
                    ],
                    start_context_char_pos=start_context_char_char,
                    end_context_char_pos=end_context_char_char,
                )
                top_valid_spans.append(span)
        top_valid_spans_batch.append(top_valid_spans)
    return top_valid_spans_batch

In [ ]:
def prepare_sentences_and_spans(
    marked_data,
) -> Tuple[List[str], List[List[Span]]]:
    """
    Prepare sentences and label spans.

    Args:
        token_seq (List[List[str]]): Tokens lists.
        label_seq (List[List[str]]): Labels lists.

    Raises:
        Exception: Validate BIO-markup.

    Returns:
        Tuple[List[str], List[List[Span]]]: Sentences and label spans.
    """
    token_seq = []
    label_seq = []
    for instance in marked_data:
      t_seq = []
      l_seq = []
      for element in instance:
        t_seq.append(element[0])
        l_seq.append(element[1])
      token_seq.append(t_seq)
      label_seq.append(l_seq)
    qa_sentences, qa_labels = [], []

    for i in trange(len(token_seq)):
        current_char_pos = 0
        qa_sent, qa_lab = [], []
        for token, label in zip(token_seq[i], label_seq[i]):
            qa_sent.append(token)
            if label != "O":
                span = Span(
                    token=token,
                    label=label,
                    start_context_char_pos=current_char_pos,
                    end_context_char_pos=current_char_pos + len(token),
                )
                qa_lab.append(span)
            current_char_pos += len(token) + 1
        qa_sentences.append(" ".join(qa_sent))
        qa_labels.append(qa_lab)

    qa_labels_v2 = []
    for qa_lab in qa_labels:
        qa_lab_v2 = []
        for span in qa_lab:
            if span.label.startswith("B-"):
                span_v2 = Span(
                    token=span.token,
                    label=span.label.split("-")[-1],
                    start_context_char_pos=span.start_context_char_pos,
                    end_context_char_pos=span.end_context_char_pos,
                )
                qa_lab_v2.append(span_v2)
            elif span.label.startswith("I-"):
                # TODO: remove duplicates and optimize
                span_v2 = Span(  # TODO: maybe use Span as dataclass
                    token=f"{span_v2.token} {span.token}",
                    label=span_v2.label,
                    start_context_char_pos=span_v2.start_context_char_pos,
                    end_context_char_pos=span.end_context_char_pos,
                )
                qa_lab_v2[-1] = span_v2
            else:
                raise ValueError(f"Unknown label: {span.label}")
        qa_labels_v2.append(qa_lab_v2)

    return qa_sentences, qa_labels_v2

In [ ]:
def compute_metrics(
    spans_true_batch: List[Span],
    spans_pred_batch_top_1: List[List[Span]],
    prompt_mapper: Dict[str, str],
) -> Dict[str, float]:
    """
    Compute NER metrics.

    Args:
        spans_true_batch (List[Span]): targets.
        spans_pred_batch_top_1 (np.ndarray): predictions.
        prompt_mapper (Dict[str, str]): prompt mapper.

    Returns:
        Dict[str, float]: metrics.
    """

    metrics = {}

    entity_mapper = {"O": 0}
    for entity_tag in prompt_mapper:
        entity_mapper[entity_tag] = len(entity_mapper)

    ner_confusion_matrix = np.zeros((len(entity_mapper), len(entity_mapper)))
    confusion_matrix_true_denominator = np.zeros(len(entity_mapper))
    confusion_matrix_pred_denominator = np.zeros(len(entity_mapper))

    for span_true, span_pred in zip(spans_true_batch, spans_pred_batch_top_1):
        span_pred = span_pred[0]  # type: ignore

        i = entity_mapper[span_true.label]
        j = entity_mapper[span_pred.label]  # type: ignore

        confusion_matrix_true_denominator[i] += 1
        confusion_matrix_pred_denominator[j] += 1

        if span_true == span_pred:
            ner_confusion_matrix[i, j] += 1

    assert (
        confusion_matrix_true_denominator.sum()
        == confusion_matrix_pred_denominator.sum()
    )

    ner_confusion_matrix_diag = np.diag(ner_confusion_matrix)

    # TODO: hide RuntimeWarning
    accuracy = np.nan_to_num(
        ner_confusion_matrix_diag.sum() / confusion_matrix_true_denominator.sum()
    )
    precision_per_entity_type = np.nan_to_num(
        ner_confusion_matrix_diag / confusion_matrix_pred_denominator
    )
    recall_per_entity_type = np.nan_to_num(
        ner_confusion_matrix_diag / confusion_matrix_true_denominator
    )
    f1_per_entity_type = np.nan_to_num(
        2
        * precision_per_entity_type
        * recall_per_entity_type
        / (precision_per_entity_type + recall_per_entity_type)
    )

    metrics["accuracy"] = accuracy

    for label_tag, idx in entity_mapper.items():
        metrics[f"precision_tag_{label_tag}"] = precision_per_entity_type[idx]
        metrics[f"recall_tag_{label_tag}"] = recall_per_entity_type[idx]
        metrics[f"f1_tag_{label_tag}"] = f1_per_entity_type[idx]

    # TODO: add micro average

    # macro average
    metrics["precision_macro"] = precision_per_entity_type.mean()
    metrics["recall_macro"] = recall_per_entity_type.mean()
    metrics["f1_macro"] = f1_per_entity_type.mean()

    # weighted average
    metrics["precision_weighted"] = np.average(
        precision_per_entity_type,
        weights=confusion_matrix_true_denominator,
    )
    metrics["recall_weighted"] = np.average(
        recall_per_entity_type,
        weights=confusion_matrix_true_denominator,
    )
    metrics["f1_weighted"] = np.average(
        f1_per_entity_type,
        weights=confusion_matrix_true_denominator,
    )

    return metrics

In [ ]:
from collections import defaultdict

import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from transformers import AutoModelForQuestionAnswering


# TODO: add metrics calculation
def train(
    n_epochs: int,
    model: AutoModelForQuestionAnswering,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    writer: SummaryWriter,
    device: torch.device,
) -> None:
    """
    Training loop.

    Args:
        n_epochs (int): number of epochs to train.
        model (AutoModelForQuestionAnswering): model.
        train_dataloader (torch.utils.data.DataLoader): train_dataloader.
        test_dataloader (torch.utils.data.DataLoader): test_dataloader.
        optimizer (torch.optim.Optimizer): optimizer.
        writer (SummaryWriter): tensorboard writer.
        device (torch.device): cpu or cuda.
    """

    for epoch in range(n_epochs):

        print(f"Epoch [{epoch+1} / {n_epochs}]\n")

        train_epoch(
            model=model,
            dataloader=train_dataloader,
            optimizer=optimizer,
            writer=writer,
            device=device,
            epoch=epoch,
        )
        evaluate_epoch(
            model=model,
            dataloader=test_dataloader,
            writer=writer,
            device=device,
            epoch=epoch,
        )


def train_epoch(
    model: AutoModelForQuestionAnswering,
    dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    writer: SummaryWriter,
    device: torch.device,
    epoch: int,
) -> None:
    """
    One training cycle (loop).

    Args:
        model (AutoModelForQuestionAnswering): QA model.
        dataloader (torch.utils.data.DataLoader): dataloader.
        optimizer (torch.optim.Optimizer): optimizer.
        writer (SummaryWriter): tensorboard writer.
        device (torch.device): cpu or cuda.
        epoch (int): number of current epochs.
    """

    model.train()

    epoch_loss = []
    batch_metrics_list = defaultdict(list)

    for i, inputs in tqdm(
        enumerate(dataloader),
        total=len(dataloader),
        desc="loop over train batches",
    ):
        optimizer.zero_grad()

        instances_batch = inputs.pop("instances")

        context_list, question_list = [], []
        for instance in instances_batch:
            context_list.append(instance.context)
            question_list.append(instance.question)

        inputs = inputs.to(device)
        offset_mapping_batch = inputs.pop("offset_mapping")

        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.item())
        writer.add_scalar(
            "batch loss / train", loss.item(), epoch * len(dataloader) + i
        )

        with torch.no_grad():
            model.eval()
            outputs_inference = model(**inputs)
            model.train()

        spans_pred_batch_top_1 = get_top_valid_spans(
            context_list=context_list,
            question_list=question_list,
            prompt_mapper=dataloader.dataset.prompt_mapper,
            inputs=inputs,
            outputs=outputs_inference,
            offset_mapping_batch=offset_mapping_batch,
            n_best_size=1,
            max_answer_length=100,  # TODO: remove hardcode
        )

        # TODO: maybe move into get_top_valid_spans
        for idx in range(len(spans_pred_batch_top_1)):
            if not spans_pred_batch_top_1[idx]:
                empty_span = Span(
                    token="",
                    label="O",  # TODO: maybe not "O" label
                    start_context_char_pos=0,
                    end_context_char_pos=0,
                )
                spans_pred_batch_top_1[idx] = [empty_span]

        spans_true_batch = [instance.answer for instance in instances_batch]

        batch_metrics = compute_metrics(
            spans_true_batch=spans_true_batch,
            spans_pred_batch_top_1=spans_pred_batch_top_1,
            prompt_mapper=dataloader.dataset.prompt_mapper,
        )

        for metric_name, metric_value in batch_metrics.items():
            batch_metrics_list[metric_name].append(metric_value)
            writer.add_scalar(
                f"batch {metric_name} / train",
                metric_value,
                epoch * len(dataloader) + i,
            )

    avg_loss = np.mean(epoch_loss)
    print(f"Train loss: {avg_loss}\n")
    writer.add_scalar("loss / train", avg_loss, epoch)

    for metric_name, metric_value_list in batch_metrics_list.items():
        metric_value = np.mean(metric_value_list)
        print(f"Train {metric_name}: {metric_value}\n")
        writer.add_scalar(f"{metric_name} / train", metric_value, epoch)


# TODO: remove train_epoch code duplicates
def evaluate_epoch(
    model: AutoModelForQuestionAnswering,
    dataloader: torch.utils.data.DataLoader,
    writer: SummaryWriter,
    device: torch.device,
    epoch: int,
) -> None:
    """
    One evaluation cycle (loop).

    Args:
        model (AutoModelForQuestionAnswering): model.
        dataloader (torch.utils.data.DataLoader): dataloader.
        writer (SummaryWriter): tensorboard writer.
        device (torch.device): cpu or cuda.
        epoch (int): number of current epochs.
    """

    model.eval()

    epoch_loss = []
    batch_metrics_list = defaultdict(list)

    with torch.no_grad():

        for i, inputs in tqdm(
            enumerate(dataloader),
            total=len(dataloader),
            desc="loop over test batches",
        ):

            instances_batch = inputs.pop("instances")

            context_list, question_list = [], []
            for instance in instances_batch:
                context_list.append(instance.context)
                question_list.append(instance.question)

            inputs = inputs.to(device)
            offset_mapping_batch = inputs.pop("offset_mapping")

            outputs = model(**inputs)
            loss = outputs.loss

            epoch_loss.append(loss.item())
            writer.add_scalar(
                "batch loss / test", loss.item(), epoch * len(dataloader) + i
            )

            spans_pred_batch_top_1 = get_top_valid_spans(
                context_list=context_list,
                question_list=question_list,
                prompt_mapper=dataloader.dataset.prompt_mapper,
                inputs=inputs,
                outputs=outputs,
                offset_mapping_batch=offset_mapping_batch,
                n_best_size=1,  # TODO: remove hardcode
                max_answer_length=100,  # TODO: remove hardcode
            )

            # TODO: maybe move into get_top_valid_spans
            for idx in range(len(spans_pred_batch_top_1)):
                if not spans_pred_batch_top_1[idx]:
                    empty_span = Span(
                        token="",
                        label="O",  # TODO: maybe not "O" label
                        start_context_char_pos=0,
                        end_context_char_pos=0,
                    )
                    spans_pred_batch_top_1[idx] = [empty_span]

            # TODO: change metrics calculation for inference time
            spans_true_batch = [instance.answer for instance in instances_batch]

            batch_metrics = compute_metrics(
                spans_true_batch=spans_true_batch,
                spans_pred_batch_top_1=spans_pred_batch_top_1,
                prompt_mapper=dataloader.dataset.prompt_mapper,
            )

            for metric_name, metric_value in batch_metrics.items():
                batch_metrics_list[metric_name].append(metric_value)
                writer.add_scalar(
                    f"batch {metric_name} / test",
                    metric_value,
                    epoch * len(dataloader) + i,
                )

        avg_loss = np.mean(epoch_loss)
        print(f"Test loss:  {avg_loss}\n")
        writer.add_scalar("loss / test", avg_loss, epoch)

        for metric_name, metric_value_list in batch_metrics_list.items():
            metric_value = np.mean(metric_value_list)
            print(f"Test {metric_name}: {metric_value}\n")
            writer.add_scalar(f"{metric_name} / test", np.mean(metric_value), epoch)

In [ ]:
from typing import Any, Dict, List, Tuple

import torch

# TODO: add documentation
class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        qa_sentences: List[str],
        qa_labels: List[List[Span]],
        prompt_mapper: Dict[str, str],
    ):
        super().__init__()
        self.prompt_mapper = prompt_mapper
        self.dataset = self._prepare_dataset(
            qa_sentences=qa_sentences,
            qa_labels=qa_labels,
        )

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, idx: int) -> Instance:
        return self.dataset[idx]

    def _prepare_dataset(
        self,
        qa_sentences: List[str],
        qa_labels: List[List[Span]],
    ) -> List[Instance]:

        dataset = []

        for sentence, labels in tqdm(
            zip(qa_sentences, qa_labels),
            desc="prepare_dataset",
        ):
            for label_tag, label_name in self.prompt_mapper.items():
                question_prompt = f"Какие {label_name}?"

                answer_list = []
                for span in labels:
                    if span.label == label_tag:
                        answer_list.append(span)

                if len(answer_list) == 0:
                    empty_span = Span(
                        token="",
                        label="O",
                        start_context_char_pos=0,
                        end_context_char_pos=0,
                    )
                    instance = Instance(
                        context=sentence,
                        question=question_prompt,
                        answer=empty_span,
                    )
                    dataset.append(instance)
                else:
                    for answer in answer_list:
                        instance = Instance(
                            context=sentence,
                            question=question_prompt,
                            answer=answer,
                        )
                        dataset.append(instance)

        return dataset


# TODO: add documentation
class Collator:
    def __init__(
        self,
        tokenizer: transformers.PreTrainedTokenizer,
        tokenizer_kwargs: Dict[str, Any],
    ):
        self.tokenizer = tokenizer
        self.tokenizer_kwargs = tokenizer_kwargs

    def __call__(
        self,
        batch: List[Instance],
    ) -> transformers.tokenization_utils_base.BatchEncoding:

        context_list = []
        question_list = []
        start_end_context_char_pos_list = []

        for instance in batch:
            context_list.append(instance.context)
            question_list.append(instance.question)
            start_end_context_char_pos_list.append(
                [
                    instance.answer.start_context_char_pos,
                    instance.answer.end_context_char_pos,
                ]
            )

        tokenized_batch = self.tokenizer(
            question_list, context_list, **self.tokenizer_kwargs
        )
        offset_mapping_batch = tokenized_batch["offset_mapping"].numpy().tolist()
        #print([len(tokenized_batch["input_ids"][0]), len(context_list[0])])
        #print(len(offset_mapping_batch[0]))
        #print(start_end_context_char_pos_list)
        assert len(offset_mapping_batch) == len(start_end_context_char_pos_list)

        # convert char boudaries to token boudaries in batch
        start_token_pos_list, end_token_pos_list = [], []
        for offset_mapping, start_end_context_char_pos, sample in zip(
            offset_mapping_batch, start_end_context_char_pos_list, batch
        ):
            # TODO: add comment
            if start_end_context_char_pos == [0, 0]:
                start_token_pos_list.append(0)
                end_token_pos_list.append(0)
            else:
                (
                    start_token_pos,
                    end_token_pos,
                ) = self.convert_char_boudaries_to_token_boudaries(
                    offset_mapping=offset_mapping,
                    start_end_context_char_pos=start_end_context_char_pos, sample=sample
                )
                #print([start_token_pos, end_token_pos])
                start_token_pos_list.append(start_token_pos)
                end_token_pos_list.append(end_token_pos)

        tokenized_batch["start_positions"] = torch.LongTensor(start_token_pos_list)
        tokenized_batch["end_positions"] = torch.LongTensor(end_token_pos_list)

        # additional
        tokenized_batch["instances"] = batch

        return tokenized_batch

    # TODO: add tests
    @staticmethod
    def convert_char_boudaries_to_token_boudaries(
        offset_mapping: List[List[int]],
        start_end_context_char_pos: List[int], sample
    ) -> Tuple[int, int]:

        start_context_char_pos, end_context_char_pos = start_end_context_char_pos
        assert end_context_char_pos >= start_context_char_pos

        done = False
        count = 0
        for ind, bond in enumerate(offset_mapping):
          if ind == 0:
            continue
          if bond == [0, 0] and offset_mapping[ind-1] != [0, 0]:
            count += 1
        res_start_token_pos = -1
        res_end_token_pos = -1
        if count == 1:
          for i, token_boudaries in enumerate(offset_mapping):
            if i == 0:
              continue
            start_token_pos, end_token_pos = token_boudaries

            if start_token_pos == start_context_char_pos:
                res_start_token_pos = i

            if end_token_pos == end_context_char_pos:
                res_end_token_pos = i  # inclusive
                done = True
                break
            if i == len(offset_mapping)-1 and res_end_token_pos == -1:
              res_end_token_pos = i-1
              done = True
              break
            if start_token_pos > start_context_char_pos and res_start_token_pos == -1:
              res_start_token_pos = i-1
            if end_token_pos > end_context_char_pos and res_end_token_pos == -1:
              res_end_token_pos = i
              done = True
              break
        else:
          special_tokens_cnt = 0
          for i, token_boudaries in enumerate(offset_mapping):

              if token_boudaries == [0, 0] and i != len(offset_mapping)-1:
                  special_tokens_cnt += 1
                  continue

              if special_tokens_cnt == 2:
                  start_token_pos, end_token_pos = token_boudaries

                  if start_token_pos == start_context_char_pos:
                      res_start_token_pos = i
                  if start_token_pos > start_context_char_pos and res_start_token_pos == -1:
                    res_start_token_pos = i-1
                  if end_token_pos == end_context_char_pos:
                      res_end_token_pos = i  # inclusive
                      done = True
                      break
                  if end_token_pos > end_context_char_pos and res_end_token_pos == -1:
                    res_end_token_pos = i
                    done = True
                    break
                  if i == len(offset_mapping)-1 and res_end_token_pos == -1:
                    res_end_token_pos = i-1
                    done = True
                    print("Catched")
                    break
        if not done:
          print(offset_mapping)
          print(start_end_context_char_pos)
          print(sample)
          print(count)
        assert done

        return res_start_token_pos, res_end_token_pos

In [ ]:
def predict(
    context: str,
    question: str,  # TODO: change with entity type
    prompt_mapper: Dict[str, str],
    model: AutoModelForQuestionAnswering,
    tokenizer: AutoTokenizer,
    tokenizer_kwargs: Dict[str, Any],
    n_best_size: int = 1,
    max_answer_length: int = 100,
) -> Instance:
    """
    Predict function.

    Args:
        context (str): context string.
        question (str): question string.
        prompt_mapper (Dict[str, str]): prompt mapper.
        model (AutoModelForQuestionAnswering): trained QaNER model.
        tokenizer (AutoTokenizer): tokenizer for trained QaNER model.
        tokenizer_kwargs (Dict[str, Any]): tokenizer kwargs.
        n_best_size (int): number of best QA answers to consider. Defaults to 1.
        max_answer_length (int): entity max length to eliminate very long entities. Defaults to 100.

    Returns:
        Instance: predicted instance.
    """

    inputs = tokenizer([question], [context], **tokenizer_kwargs).to(model.device)
    offset_mapping_batch = inputs.pop("offset_mapping")

    with torch.no_grad():
        outputs = model(**inputs)

    spans_pred_batch_top = get_top_valid_spans(
        context_list=[context],
        question_list=[question],
        prompt_mapper=prompt_mapper,
        inputs=inputs,
        outputs=outputs,
        offset_mapping_batch=offset_mapping_batch,
        n_best_size=n_best_size,
        max_answer_length=max_answer_length,
    )[0]

    # TODO: validate it
    spans_pred_batch_top = [span for span in spans_pred_batch_top if span]

    for predicted_answer_span in spans_pred_batch_top:
        start_pos = predicted_answer_span.start_context_char_pos
        end_pos = predicted_answer_span.end_context_char_pos
        assert predicted_answer_span.token == context[start_pos:end_pos]

    prediction = Instance(
        context=context,
        question=question,
        answer=spans_pred_batch_top,
    )

    return prediction

#QANER training

In [ ]:
log_dir = "/content/logs"
bert_model_name = "Stacy123/rubert_tiny_qa_kontur"
batch_size = 32
learning_rate = 1e-5
seed = 42
path_to_save_model = "/content/model"
n_epochs = 100
max_len = 512

In [ ]:
set_global_seed(seed)

In [ ]:
import json
with open("/content/prompt_mapper.json", mode="r", encoding="utf-8") as fp:
      prompt_mapper = json.load(fp)

In [ ]:
all_qa_sentences, all_qa_labels = prepare_sentences_and_spans(marked)

100%|██████████| 873/873 [00:00<00:00, 899.44it/s]


In [ ]:
all_qa_labels[3]

[Span(token='Выполнение работ по гнутью и резке арматурной стали на ручных, электромеханических и электрических станках.', label='requirements', start_context_char_pos=0, end_context_char_pos=107)]

In [ ]:
maxlen = 3000
qa_sentences = []
qa_labels = []
for sentence, labels in zip(all_qa_sentences, all_qa_labels):
  if len(sentence) <= maxlen:
    qa_sentences.append(sentence)
    qa_labels.append(labels)

In [ ]:
len(qa_sentences)

872

In [ ]:
train_qa_sentences = qa_sentences[:750]
test_qa_sentences = qa_sentences[750:]
train_qa_labels = qa_labels[:750]
test_qa_labels = qa_labels[750:]

In [ ]:
train_dataset = Dataset(
        qa_sentences=train_qa_sentences,
        qa_labels=train_qa_labels,
        prompt_mapper=prompt_mapper,
    )

prepare_dataset: 750it [00:00, 190350.24it/s]


In [ ]:
test_dataset = Dataset(
        qa_sentences=test_qa_sentences,
        qa_labels=test_qa_labels,
        prompt_mapper=prompt_mapper,
    )

prepare_dataset: 122it [00:00, 109035.82it/s]


In [ ]:
train_dataset[3]

Instance(context='Гнутье арматурной стали на механических станках при количестве отгибов на одном стержне более четырех. Сборка и установка сеток и плоских...', question='Какие требования к соискателю?', answer=Span(token='', label='O', start_context_char_pos=0, end_context_char_pos=0))

In [ ]:
maxlen = 0
for sample in test_dataset:
  if len(sample.context) > maxlen:
    maxlen = len(sample.context)
print(maxlen)

2154


In [ ]:
from torch.utils.tensorboard import SummaryWriter
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import DistilBertTokenizerFast, BertTokenizer

In [ ]:
writer = SummaryWriter(log_dir=log_dir)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#tokenizer = BertTokenizer.from_pretrained(bert_model_name)
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
#tokenizer = DistilBertTokenizerFast.from_pretrained(bert_model_name)
model = AutoModelForQuestionAnswering.from_pretrained(bert_model_name).to(device)

tokenizer_kwargs = {
    "max_length": 512,
    "truncation": "only_second",
    "padding": True,
    "return_tensors": "pt",
    "return_offsets_mapping": True,
}

collator = Collator(
    tokenizer=tokenizer,
    tokenizer_kwargs=tokenizer_kwargs,
)

train_dataloader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collator,
)

test_dataloader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collator,
)

In [ ]:
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

In [ ]:
train(
    n_epochs=n_epochs,
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=optimizer,
    writer=writer,
    device=device,
)

model.save_pretrained(path_to_save_model)
tokenizer.save_pretrained(path_to_save_model)

Epoch [1 / 100]



loop over train batches:   0%|          | 0/71 [00:00<?, ?it/s]<ipython-input-34-682b55d089f1>:52: RuntimeWarning: invalid value encountered in true_divide
  ner_confusion_matrix_diag / confusion_matrix_pred_denominator
<ipython-input-34-682b55d089f1>:55: RuntimeWarning: invalid value encountered in true_divide
  ner_confusion_matrix_diag / confusion_matrix_true_denominator
<ipython-input-34-682b55d089f1>:58: RuntimeWarning: invalid value encountered in true_divide
  2
loop over train batches:  24%|██▍       | 17/71 [00:05<00:16,  3.32it/s]

Catched


loop over train batches: 100%|██████████| 71/71 [00:23<00:00,  3.02it/s]


Train loss: 4.687721396835757

Train accuracy: 0.3316901408450704

Train precision_tag_O: 0.3743602611448339

Train recall_tag_O: 0.9474589768603852

Train f1_tag_O: 0.5321970414402007

Train precision_tag_requirements: 0.0

Train recall_tag_requirements: 0.0

Train f1_tag_requirements: 0.0

Train precision_tag_terms: 0.0

Train recall_tag_terms: 0.0

Train f1_tag_terms: 0.0

Train precision_tag_notes: 0.0

Train recall_tag_notes: 0.0

Train f1_tag_notes: 0.0

Train precision_macro: 0.09359006528620847

Train recall_macro: 0.2368647442150963

Train f1_macro: 0.1330492603600502

Train precision_weighted: 0.1359143036045103

Train recall_weighted: 0.3316901408450704

Train f1_weighted: 0.19135316092751747



...

loop over train batches: 100%|██████████| 71/71 [00:25<00:00,  2.81it/s]


Train loss: 0.31825710917022865

Train accuracy: 0.32411971830985914

Train precision_tag_O: 0.7857447004489475

Train recall_tag_O: 0.9269046669523059

Train f1_tag_O: 0.8445233519297861

Train precision_tag_requirements: 0.0

Train recall_tag_requirements: 0.0

Train f1_tag_requirements: 0.0

Train precision_tag_terms: 0.0010834236186348864

Train recall_tag_terms: 0.0010060362173038228

Train f1_tag_terms: 0.0010432968179447054

Train precision_tag_notes: 0.0

Train recall_tag_notes: 0.0

Train f1_tag_notes: 0.0

Train precision_macro: 0.1967070310168956

Train recall_macro: 0.23197767579240242

Train f1_macro: 0.21139166218693273

Train precision_weighted: 0.28128186485413587

Train recall_weighted: 0.32411971830985914

Train f1_weighted: 0.299433467839833



loop over test batches: 100%|██████████| 12/12 [00:01<00:00, 10.99it/s]


Test loss:  1.4855514168739319

Test accuracy: 0.3363095238095238

Test precision_tag_O: 0.7101901151823754

Test recall_tag_O: 0.9228729603729605

Test f1_tag_O: 0.799549575029335

Test precision_tag_requirements: 0.0

Test recall_tag_requirements: 0.0

Test f1_tag_requirements: 0.0

Test precision_tag_terms: 0.0

Test recall_tag_terms: 0.0

Test f1_tag_terms: 0.0

Test precision_tag_notes: 0.0

Test recall_tag_notes: 0.0

Test f1_tag_notes: 0.0

Test precision_macro: 0.17754752879559385

Test recall_macro: 0.23071824009324013

Test f1_macro: 0.19988739375733375

Test precision_weighted: 0.2598111454855221

Test recall_weighted: 0.3363095238095238

Test f1_weighted: 0.29196000571715053



('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.txt',
 '/content/model/added_tokens.json',
 '/content/model/tokenizer.json')

In [ ]:
sample = test_dataset[3]
sample

Instance(context='Описание работодателя: компания АО ;Ю-ПИ-СИ; занимается управлением бизнес-центром Требования: Среднее профессиональное образование, опыт работы не менее 2 лет. Обязанности: Обеспечивать исправное состояние, безаварийную и надежную работу обслуживаемых систем отопления, водоснабжения, канализации и водостоков, правильную их эксплуатацию, своевременный и качественный ремонт. Условия: Работа в стабильной организации. Испытательный срок - 3 месяца. Оформление по ТК РФ. 40-часовая рабочая неделя, 8-часовой рабочий день. Начало работы: 8:30. Окончание: 17:00.', question='Какие требования к соискателю?', answer=Span(token='Среднее профессиональное образование, опыт работы не менее 2 лет.', label='requirements', start_context_char_pos=95, end_context_char_pos=160))

In [ ]:
predict(sample.context, sample.question, prompt_mapper, model, tokenizer, tokenizer_kwargs, n_best_size=2)

Instance(context='Описание работодателя: компания АО ;Ю-ПИ-СИ; занимается управлением бизнес-центром Требования: Среднее профессиональное образование, опыт работы не менее 2 лет. Обязанности: Обеспечивать исправное состояние, безаварийную и надежную работу обслуживаемых систем отопления, водоснабжения, канализации и водостоков, правильную их эксплуатацию, своевременный и качественный ремонт. Условия: Работа в стабильной организации. Испытательный срок - 3 месяца. Оформление по ТК РФ. 40-часовая рабочая неделя, 8-часовой рабочий день. Начало работы: 8:30. Окончание: 17:00.', question='Какие требования к соискателю?', answer=[Span(token='Среднее', label='requirements', start_context_char_pos=95, end_context_char_pos=102), Span(token='опыт', label='requirements', start_context_char_pos=133, end_context_char_pos=137)])